# Characteristic Evaluation

## Preprocess

In [1]:
import pandas as pd
import numpy as np

In [7]:
ratio_df=pd.read_csv('data/CHN_ratios.csv',index_col=0)
ratio_df=ratio_df.dropna()
def multiplier(num):
    out=np.nan
    if num[-1]=='B':
        out=float(num[:-1])*1000
    elif num[-1]=='T':
        out=float(num[:-1])*1000*1000
    elif num[-1]=='M':
        out=float(num[:-1])
    return out
ratio_df['mkt_cap']=ratio_df['mkt_cap'].map(multiplier)
ratio_df['company']=ratio_df['company'].map(lambda x: x[:-2]+'SH' if x[-2:]=='SS' else x)
ratio_df['profit_m']=ratio_df['profit_m'].map(lambda x: float(x.strip('%'))/100)
ratio_df['roa']=ratio_df['roa'].map(lambda x: float(x.strip('%'))/100)
ratio_df['roe']=ratio_df['roe'].map(lambda x: float(x.strip('%'))/100)
charic_names=list(ratio_df.columns[2:])
ratio_df

,company,mkt_cap,pb_ratio,beta,profit_m,roa,roe
1,300467.SZ,3710.0,1.58,-0.15,-2.6326,0.0064,-0.6957
2,601997.SH,24650.0,0.75,0.72,0.5697,0.0114,0.1589
4,600641.SH,14750.0,2.35,0.78,0.2569,0.0441,0.0709
6,002653.SZ,21240.0,9.51,0.31,0.1553,0.0419,0.2410
8,600569.SH,6150.0,0.61,0.80,-0.0050,0.0092,-0.0170
...,...,...,...,...,...,...,...
3763,000661.SZ,110920.0,13.72,0.61,0.2689,0.1500,0.3058
3764,601288.SH,1160000.0,0.63,0.85,0.4286,0.0086,0.1141
3766,002931.SZ,2110.0,4.03,0.03,0.1369,0.0495,0.1129
3767,603136.SH,2520.0,2.82,0.62,0.1891,0.0653,0.0832


Scope out outliers: only include the samples within 5 sigmas

In [8]:
ratio_mean=ratio_df.mean()
ratio_std=ratio_df.std()
for i in range(ratio_df.shape[1]-2):
    ratio_df=ratio_df[(ratio_df[charic_names[i]]>ratio_mean[i+1]-5*ratio_std[i+1])&(ratio_df[charic_names[i]]<ratio_mean[i+1]+5*ratio_std[i+1])]
ratio_df

,company,mkt_cap,pb_ratio,beta,profit_m,roa,roe
2,601997.SH,24650.0,0.75,0.72,0.5697,0.0114,0.1589
4,600641.SH,14750.0,2.35,0.78,0.2569,0.0441,0.0709
6,002653.SZ,21240.0,9.51,0.31,0.1553,0.0419,0.2410
8,600569.SH,6150.0,0.61,0.80,-0.0050,0.0092,-0.0170
9,000797.SZ,5300.0,0.93,1.27,0.0688,0.0178,0.0626
...,...,...,...,...,...,...,...
3763,000661.SZ,110920.0,13.72,0.61,0.2689,0.1500,0.3058
3764,601288.SH,1160000.0,0.63,0.85,0.4286,0.0086,0.1141
3766,002931.SZ,2110.0,4.03,0.03,0.1369,0.0495,0.1129
3767,603136.SH,2520.0,2.82,0.62,0.1891,0.0653,0.0832


In [13]:
cluster_df=pd.read_csv('all_cluster_chn_seg.csv',index_col=0)
cluster_names=list(cluster_df.columns[1:])
all_df=cluster_df.merge(ratio_df,how='inner',on='company')
all_df

,company,industry,paper_class,kmean_class,bert,bag of words,pvdm 50 length,pvdbow 50 length,pvdm 100 length,pvdbow 100 length,...,lsi 800 length,lsi 900 length,lsi 1000 length,lsi 1100 length,mkt_cap,pb_ratio,beta,profit_m,roa,roe
0,600000.SH,44,13,16,53,13,23,13,27,49,...,41,27,35,18,297920.0,0.58,0.40,0.5188,0.0087,0.1120
1,600004.SH,20,12,20,53,12,17,13,23,49,...,36,11,24,3,25970.0,1.64,1.24,0.0966,0.0199,0.0445
2,600007.SH,65,36,54,53,31,43,13,50,49,...,18,29,16,47,13300.0,1.81,0.59,0.2755,0.0754,0.1343
3,600008.SH,63,1,20,53,50,17,13,50,49,...,36,11,9,41,17170.0,0.99,1.04,0.0624,0.0213,0.0414
4,600009.SH,20,12,20,53,12,41,13,54,49,...,58,11,56,3,117180.0,3.66,0.76,0.3793,0.0690,0.1256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,300738.SZ,49,54,44,53,41,17,13,10,49,...,7,6,35,14,6720.0,9.80,-0.13,0.1651,0.0544,0.2327
2778,300739.SZ,53,46,12,53,12,17,13,31,49,...,52,14,47,3,4730.0,3.65,0.25,0.1194,0.0360,0.1049
2779,300740.SZ,37,11,1,53,41,48,13,32,49,...,41,45,24,40,3050.0,2.53,0.00,0.0116,0.0072,0.0206
2780,300741.SZ,2,8,43,53,50,52,13,6,49,...,40,14,56,31,19430.0,2.50,0.06,0.5539,0.0759,0.1394


## Evaluation

In [5]:
# mkt_cap weighted average
eval_df=pd.DataFrame(columns=['clustering']+charic_names)
total_comp=all_df.shape[0]
charic_total=[]
for r in range(len(charic_names)):
    charic_total.append(ratio_df['mkt_cap'].dot(ratio_df.iloc[:,r+2]))
charic_total=charic_total/ratio_df['mkt_cap'].sum()
cl_results=[]
for cl in range(len(cluster_names)):
    cl_res=[cluster_names[cl]]
    for r in range(len(charic_names)):
        clr_df=all_df.loc[:,[cluster_names[cl],charic_names[r],'mkt_cap']]
        clr_df['sumpro']=clr_df[charic_names[r]]*clr_df['mkt_cap']
        clr_df=clr_df.groupby(cluster_names[cl]).agg({charic_names[r]:'count','mkt_cap':'sum','sumpro':'sum'})
        clr_df=clr_df[clr_df[charic_names[r]]>=5]
        clr_df['sigma']=clr_df[charic_names[r]]/np.sum(clr_df[charic_names[r]])*(clr_df['sumpro']/clr_df['mkt_cap']-charic_total[r])**2
        clr=np.sqrt(clr_df['sigma'].sum())
        cl_res.append(clr)
    cl_results.append(cl_res)
eval_df=eval_df.append(pd.DataFrame(cl_results,columns=eval_df.columns))
eval_df['average']=eval_df.mean(1)
eval_df.set_index('clustering',inplace=True)
eval_df=eval_df.sort_values('average', ascending=False)
eval_df

,pb_ratio,beta,profit_m,roa,roe,average
clustering,,,,,,
paper_class,2.603119,0.203967,0.119993,0.026543,0.057697,0.602264
industry,2.570319,0.191177,0.121894,0.026817,0.058487,0.593739
kmean_class,2.502374,0.193221,0.113279,0.023038,0.056713,0.577725
lsi 500 length,2.244430,0.113197,0.096857,0.022083,0.034256,0.502165
pvdm 150 length,2.111500,0.152096,0.105350,0.020765,0.044452,0.486832
pvdm 350 length,2.084813,0.144841,0.092095,0.021816,0.042931,0.477299
pvdm 300 length,2.003381,0.157801,0.096131,0.024858,0.046695,0.465773
pvdm 100 length,1.946729,0.165567,0.100099,0.018007,0.042118,0.454504
lsi 400 length,1.948267,0.130997,0.088269,0.024932,0.035539,0.445601


In [14]:
# arithmatic average
eval_df=pd.DataFrame(columns=['clustering']+charic_names)
total_comp=all_df.shape[0]
charic_total=[]
for r in range(len(charic_names)):
    charic_total.append(ratio_df.iloc[:,r+2].mean())
cl_results=[]
for cl in range(len(cluster_names)):
    cl_res=[cluster_names[cl]]
    for r in range(len(charic_names)):
        clr_df=all_df.loc[:,[cluster_names[cl],charic_names[r]]]
        clr_df=clr_df.groupby(cluster_names[cl]).agg(['count','mean'])
        clr_df=clr_df[clr_df[(charic_names[r],'count')]>=5]
        clr_df['sigma']=clr_df[(charic_names[r],'count')]/np.sum(clr_df[(charic_names[r],'count')])*(clr_df[(charic_names[r],'mean')]-charic_total[r])**2
        clr=np.sqrt(clr_df['sigma'].sum())
        cl_res.append(clr)
    cl_results.append(cl_res)
eval_df=eval_df.append(pd.DataFrame(cl_results,columns=eval_df.columns))
eval_df['average']=eval_df.mean(1)
eval_df.set_index('clustering',inplace=True)
eval_df=eval_df.sort_values('average', ascending=False)
eval_df

,pb_ratio,beta,profit_m,roa,roe,average
clustering,,,,,,
industry,1.129918,0.116859,0.061735,0.011193,0.031461,0.270233
kmean_class,1.013745,0.096144,0.054986,0.010350,0.031784,0.241402
paper_class,0.965938,0.096787,0.045033,0.009848,0.025995,0.228720
lsi 600 length,0.552521,0.057332,0.045743,0.005605,0.020262,0.136293
lsi 500 length,0.561250,0.065283,0.021941,0.005445,0.017393,0.134262
lsi 300 length,0.543980,0.065443,0.034428,0.006648,0.020328,0.134165
lsi 800 length,0.522893,0.045934,0.031594,0.005266,0.018250,0.124787
lsi 1100 length,0.507269,0.061710,0.030533,0.004594,0.018757,0.124573
lsi 400 length,0.485611,0.066264,0.037673,0.005692,0.019048,0.122858
